In [2]:
!pip install splinter
!pip install bs4

In [3]:
from splinter import Browser
from bs4 import BeautifulSoup
import re
import time

## For Mac

In [12]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

# visit the NASA mars news site 
url = "https://mars.nasa.gov/news/"

browser.visit(url)

#optional delay for loading the page
browser.is_element_present_by_css("ul.item_list li.slide", wait_time = 1)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

slide_elem = soup.select_one("ul.item_list li.slide")

slide_elem.find("div", class_="content_title")

<div class="content_title"><a href="/news/8647/nasas-curiosity-keeps-rolling-as-team-operates-rover-from-home/" target="_self">NASA's Curiosity Keeps Rolling As Team Operates Rover From Home</a></div>

In [13]:
news_title = slide_elem.find("div", class_= "content_title").get_text()
news_title

"NASA's Curiosity Keeps Rolling As Team Operates Rover From Home"

In [16]:
news_p = slide_elem.find("div", class_="article_teaser_body").get_text()
news_p

'The team has learned to meet new challenges as they work remotely on the Mars mission.'

## JPL Space Images Featured Images

In [28]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)

# Find and click the full image button
full_image_elem = browser.find_by_id('full_image')
full_image_elem.click()

# Find the more info button and click it
browser.is_element_present_by_text('more info', wait_time = 1)
more_info_elem = browser.links.find_by_partial_text('more info')
more_info_elem.click()

# Parse the resulting HTML with soup 
html = browser.html
img_soup = BeautifulSoup(html, 'html.parser')

# Find the relative iamge url 
img_url_rel = img_soup.select_one('figure.lede a img').get('src')
img_url_rel

img_url = f'https://www.jpl.nasa.gov{img_url_rel}'
img_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA19913_hires.jpg'

## Mars Weather

In [30]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)
time.sleep(5)

html = browser.html
weather_soup = BeautifulSoup(html, 'html.parser')

mars_weather_tweet = weather_soup.find('div', attrs={'class':'tweets','data-name':'Mars Weather'})

try:
    mars_weather = mars_weather_tweet.find('p', 'tweet-text').get_text()
    mars_weather
except AttributeError:
    pattern = re.compile(r'sol')
    mars_weather = weather_soup.find('span', text=pattern).text
    mars_weather
mars_weather

'InSight sol 492 (2020-04-15) low -94.4ºC (-137.9ºF) high -5.5ºC (22.2ºF)\nwinds from the SW at 5.1 m/s (11.5 mph) gusting to 14.9 m/s (33.3 mph)\npressure at 6.60 hPa'

## Mars Hemisphere

In [33]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

browser.visit(url)

hemisphere_image_urls = []
links = browser.find_by_css('a.product-item h3')
print(links)

for i in range(len(links)):
    hemisphere = {}
    browser.find_by_css("a.product-item h3")[i].click()
    sample_elem = browser.find_link_by_text('Sample').first
    hemisphere['img_url'] = sample_elem['href']
    hemisphere['title'] = browser.find_by_css('h2.title').text
    
    hemisphere_image_urls.append(hemisphere)
    browser.back()

/Users/meganl/opt/anaconda3/lib/python3.7/site-packages/splinter/driver/webdriver/__init__.py:536: FutureWarning: browser.find_link_by_text is deprecated. Use browser.links.find_by_text instead.
  FutureWarning,


In [34]:
hemisphere_image_urls

[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

## Mars Facts

In [35]:
# !pip install --upgrade certifi

#cd /Applications/Python \3.7/
#./Install\ Certificates.command

     |████████████████████████████████| 163kB 1.4MB/s eta 0:00:01
  Found existing installation: certifi 2019.9.11
    Uninstalling certifi-2019.9.11:
      Successfully uninstalled certifi-2019.9.11


In [36]:
import pandas as pd
df = pd.read_html("https://space-facts.com/mars/")[0]
df.columns=['description', 'value']
df.set_index('description', inplace = True)
df

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [37]:
df.to_html()
browser.quit()